# Debug of singleday_factors_annotated

In [4]:
"""Functions for plotting tca decomp."""
import os
import flow
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import tensortools as tt
import seaborn as sns
import pandas as pd
from copy import deepcopy
from cascade import df
from cascade import tca
from cascade import paths

### For some reason annotated plots do not seem to show diengaged trials. This points to either a problem in the plotting or the metadata itself. Debug of the last doay should work, as it has disengaged data.

In [15]:
trace_type='zscore_day'
method='ncp_bcd'
cs=''
warp=False
word='dame'
extra_col=4
alpha=0.6
plot_running=True
filetype='pdf'
verbose=False
mouse = 'OA27'
exclude_tags=('orientation_mapping', 'contrast', 'retinotopy')      

plot_options = {
      'cp_als': {
        'line_kw': {
          'color': 'red',
          'label': 'cp_als',
        },
        'scatter_kw': {
          'color': 'green',
          'alpha': 0.5,
        },
        'bar_kw': {
          'color': 'blue',
          'alpha': 0.5,
        },
      },
      'ncp_hals': {
        'line_kw': {
          'color': 'red',
          'label': 'ncp_hals',
        },
        'scatter_kw': {
          'color': 'green',
          'alpha': 0.5,
        },
        'bar_kw': {
          'color': 'blue',
          'alpha': 0.5,
        },
      },
      'ncp_bcd': {
        'line_kw': {
          'color': 'red',
          'label': 'ncp_bcd',
        },
        'scatter_kw': {
          'color': 'green',
          'alpha': 0.5,
        },
        'bar_kw': {
          'color': 'blue',
          'alpha': 0.5,
        },
      },
    }

pars = {'trace_type': trace_type, 'cs': cs, 'warp': warp}

days = flow.metadata.DateSorter.frommeta(mice=[mouse], tags=None)

# for day1 in days:
day1 = days[-1]

# load dir
load_dir = paths.tca_path(mouse, 'single', pars=pars, word=word)
tensor_path = os.path.join(load_dir, str(day1.mouse) + '_' + str(day1.date)
                           + '_single_decomp_' + str(trace_type) + '.npy')
meta_path = os.path.join(load_dir, str(day1.mouse) + '_' + str(day1.date)
                         + '_df_single_meta.pkl')
# if not os.path.isfile(tensor_path): continue
# if not os.path.isfile(meta_path): continue

# save dir
save_dir = paths.tca_plots(mouse, 'single', pars=pars, word=word)
save_dir = os.path.join(save_dir, 'factors annotated')
if not os.path.isdir(save_dir): os.mkdir(save_dir)
date_dir = os.path.join(save_dir, str(day1.date) + ' ' + method)
if not os.path.isdir(date_dir): os.mkdir(date_dir)

# load your data
ensemble = np.load(tensor_path)
ensemble = ensemble.item()
meta = pd.read_pickle(meta_path)
orientation = meta['orientation']
trial_num = np.arange(0, len(orientation))
condition = meta['condition']
trialerror = meta['trialerror']
hunger = meta['hunger']
speed = meta['speed']

# merge hunger and tag info for plotting hunger
tags = meta['tag']
hunger[tags == 'disengaged'] = 'disengaged'

/Users/kelly_mcguire/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
hunger

mouse  date    run  trial_idx
OA27   170404  2    0                hungry
                    1                hungry
                    2                hungry
                    3                hungry
                    4                hungry
                    5                hungry
                    6                hungry
                    7                hungry
                    8                hungry
                    9                hungry
                    10               hungry
                    11               hungry
                    12               hungry
                    13               hungry
                    14               hungry
                    15               hungry
                    16               hungry
                    17               hungry
                    18               hungry
                    19               hungry
                    20               hungry
                    21               hungry
  

In [ ]:
# sort neuron factors by component they belong to most
sort_ensemble, my_sorts = tca._sortfactors(ensemble[method])

for r in sort_ensemble.results:

    U = sort_ensemble.results[r][0].factors

    fig, axes = plt.subplots(U.rank, U.ndim + extra_col, figsize=(9 + extra_col, U.rank))
    figt = tt.plot_factors(U, plots=['bar', 'line', 'scatter'],
                    axes=None,
                    fig=fig,
                    scatter_kw=plot_options[method]['scatter_kw'],
                    line_kw=plot_options[method]['line_kw'],
                    bar_kw=plot_options[method]['bar_kw'])
    ax = figt[0].axes
    ax[0].set_title('Neuron factors')
    ax[1].set_title('Temporal factors')
    ax[2].set_title('Trial factors')

    # reshape for easier indexing
    ax = np.array(ax).reshape((U.rank, -1))

    for col in range(3, 3+extra_col):
        for i in range(U.rank):

            # get axis values
            y_lim = ax[i, 2].get_ylim()
            x_lim = ax[i, 2].get_xlim()
            y_ticks = ax[i, 2].get_yticks()
            y_tickl = ax[i, 2].get_yticklabels()
            x_ticks = ax[i, 2].get_xticks()
            x_tickl = ax[i, 2].get_xticklabels()

            # running
            if plot_running:
                scale_by = np.nanmax(speed)/y_lim[1]
                if not np.isnan(scale_by):
                    ax[i, col].plot(np.array(speed.tolist())/scale_by, color=[1, 0.1, 0.6, 0.2])
                    # , label='speed')

            # Orientation - main variable to plot
            if col == 3:
                ori_vals = [0, 135, 270]
                color_vals = [[0.28, 0.68, 0.93, alpha], [0.84, 0.12, 0.13, alpha],
                              [0.46, 0.85, 0.47, alpha]]
                for k in range(0, 3):
                    ax[i, col].plot(trial_num[orientation == ori_vals[k]],
                                    U.factors[2][orientation == ori_vals[k], i], 'o',
                                    label=str(ori_vals[k]), color=color_vals[k], markersize=2)
                if i == 0:
                    ax[i, col].set_title('Orientation')
                    ax[i, col].legend(bbox_to_anchor=(0.5,1.02), loc='lower center',
                                      borderaxespad=2.5)
            elif col == 4:
                cs_vals = ['plus', 'minus', 'neutral']
                cs_labels = ['plus', 'minus', 'neutral']
                color_vals = [[0.46, 0.85, 0.47, alpha], [0.84, 0.12, 0.13, alpha],
                              [0.28, 0.68, 0.93, alpha]]
                col = 4
                for k in range(0, 3):
                    ax[i, col].plot(trial_num[condition == cs_vals[k]],
                                    U.factors[2][condition == cs_vals[k], i], 'o',
                                    label=str(cs_labels[k]), color=color_vals[k], markersize=2)
                if i == 0:
                    ax[i, col].set_title('Condition')
                    ax[i, col].legend(bbox_to_anchor=(0.5,1.02), loc='lower center',
                                      borderaxespad=2.5)
            elif col == 5:
                trialerror_vals = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
                trialerror_labels = ['hit',
                                     'miss',
                                     'neutral correct reject',
                                     'neutral false alarm',
                                     'minus correct reject',
                                     'minus false alarm',
                                     'blank correct reject',
                                     'blank false alarm',
                                     'pav early licking',
                                     'pav late licking',]
                for k in trialerror_vals:
                    ax[i, col].plot(trial_num[trialerror == trialerror_vals[k]],
                                    U.factors[2][trialerror == trialerror_vals[k], i], 'o',
                                    label=str(trialerror_labels[k]), alpha=0.8, markersize=2)
                if i == 0:
                    ax[i, col].set_title('Trialerror')
                    ax[i, col].legend(bbox_to_anchor=(0.5, 1.02), loc='lower center',
                                      borderaxespad=2.5)

            elif col == 6:
                h_vals = ['hungry', 'sated', 'disengaged']
                h_labels = ['hungry', 'sated', 'disengaged']
                color_vals = [[1, 0.6, 0.3, alpha], [0.7, 0.9, 0.4, alpha],
                              [0.6, 0.5, 0.6, alpha], [0.0, 0.9, 0.4, alpha]]
                for k in range(0, 3):
                    ax[i, col].plot(trial_num[hunger == h_vals[k]],
                                    U.factors[2][hunger == h_vals[k], i], 'o',
                                    label=str(h_labels[k]), color=color_vals[k], markersize=2)
                if i == 0:
                    ax[i, col].set_title('State')
                    ax[i, col].legend(bbox_to_anchor=(0.5, 1.02), loc='lower center',
                                      borderaxespad=2.5)

            # set axes labels
            ax[i, col].set_yticks(y_ticks)
            ax[i, col].set_yticklabels(y_tickl)
            ax[i, col].set_ylim(y_lim)
            ax[i, col].set_xlim(x_lim)

            # format axes
            ax[i, col].locator_params(nbins=4)
            ax[i, col].spines['top'].set_visible(False)
            ax[i, col].spines['right'].set_visible(False)
            ax[i, col].xaxis.set_tick_params(direction='out')
            ax[i, col].yaxis.set_tick_params(direction='out')
            ax[i, col].yaxis.set_ticks_position('left')
            ax[i, col].xaxis.set_ticks_position('bottom')

            # remove xticks on all but bottom row
            if i + 1 != U.rank:
                plt.setp(ax[i, col].get_xticklabels(), visible=False)

            if col == 3:
                ax[i, 0].set_ylabel('Component #' + str(i+1), rotation=0,
                                    labelpad=45, verticalalignment='center',
                                    fontstyle='oblique')

    if filetype.lower() == 'pdf':
        suffix = '.pdf'
    elif filetype.lower() == 'eps':
        suffix = '.eps'
    else:
        suffix = '.png'
    plt.savefig(os.path.join(date_dir, 'rank_' + str(int(r)) + suffix),
                             bbox_inches='tight')
    if verbose:
        plt.show()
    plt.close()


In [18]:
for day1 in days:
#     day1 = days[-1]

    # load dir
    load_dir = paths.tca_path(mouse, 'single', pars=pars, word=word)
    tensor_path = os.path.join(load_dir, str(day1.mouse) + '_' + str(day1.date)
                               + '_single_decomp_' + str(trace_type) + '.npy')
    meta_path = os.path.join(load_dir, str(day1.mouse) + '_' + str(day1.date)
                             + '_df_single_meta.pkl')
    if not os.path.isfile(tensor_path): continue
    if not os.path.isfile(meta_path): continue

    # save dir
    save_dir = paths.tca_plots(mouse, 'single', pars=pars, word=word)
    save_dir = os.path.join(save_dir, 'factors annotated')
    if not os.path.isdir(save_dir): os.mkdir(save_dir)
    date_dir = os.path.join(save_dir, str(day1.date) + ' ' + method)
    if not os.path.isdir(date_dir): os.mkdir(date_dir)

    # load your data
    ensemble = np.load(tensor_path)
    ensemble = ensemble.item()
    meta = pd.read_pickle(meta_path)
    orientation = meta['orientation']
    trial_num = np.arange(0, len(orientation))
    condition = meta['condition']
    trialerror = meta['trialerror']
    hunger = meta['hunger']
    speed = meta['speed']

    # merge hunger and tag info for plotting hunger
    tags = meta['tag']
    hunger[tags == 'disengaged'] = 'disengaged'

    # sort neuron factors by component they belong to most
    sort_ensemble, my_sorts = tca._sortfactors(ensemble[method])

    for r in sort_ensemble.results:

        U = sort_ensemble.results[r][0].factors

        fig, axes = plt.subplots(U.rank, U.ndim + extra_col, figsize=(9 + extra_col, U.rank))
        figt = tt.plot_factors(U, plots=['bar', 'line', 'scatter'],
                        axes=None,
                        fig=fig,
                        scatter_kw=plot_options[method]['scatter_kw'],
                        line_kw=plot_options[method]['line_kw'],
                        bar_kw=plot_options[method]['bar_kw'])
        ax = figt[0].axes
        ax[0].set_title('Neuron factors')
        ax[1].set_title('Temporal factors')
        ax[2].set_title('Trial factors')

        # reshape for easier indexing
        ax = np.array(ax).reshape((U.rank, -1))

        for col in range(3, 3+extra_col):
            for i in range(U.rank):

                # get axis values
                y_lim = ax[i, 2].get_ylim()
                x_lim = ax[i, 2].get_xlim()
                y_ticks = ax[i, 2].get_yticks()
                y_tickl = ax[i, 2].get_yticklabels()
                x_ticks = ax[i, 2].get_xticks()
                x_tickl = ax[i, 2].get_xticklabels()

                # running
                if plot_running:
                    scale_by = np.nanmax(speed)/y_lim[1]
                    if not np.isnan(scale_by):
                        ax[i, col].plot(np.array(speed.tolist())/scale_by, color=[1, 0.1, 0.6, 0.2])
                        # , label='speed')

                # Orientation - main variable to plot
                if col == 3:
                    ori_vals = [0, 135, 270]
                    color_vals = [[0.28, 0.68, 0.93, alpha], [0.84, 0.12, 0.13, alpha],
                                  [0.46, 0.85, 0.47, alpha]]
                    for k in range(0, 3):
                        ax[i, col].plot(trial_num[orientation == ori_vals[k]],
                                        U.factors[2][orientation == ori_vals[k], i], 'o',
                                        label=str(ori_vals[k]), color=color_vals[k], markersize=2)
                    if i == 0:
                        ax[i, col].set_title('Orientation')
                        ax[i, col].legend(bbox_to_anchor=(0.5,1.02), loc='lower center',
                                          borderaxespad=2.5)
                elif col == 4:
                    cs_vals = ['plus', 'minus', 'neutral']
                    cs_labels = ['plus', 'minus', 'neutral']
                    color_vals = [[0.46, 0.85, 0.47, alpha], [0.84, 0.12, 0.13, alpha],
                                  [0.28, 0.68, 0.93, alpha]]
                    col = 4
                    for k in range(0, 3):
                        ax[i, col].plot(trial_num[condition == cs_vals[k]],
                                        U.factors[2][condition == cs_vals[k], i], 'o',
                                        label=str(cs_labels[k]), color=color_vals[k], markersize=2)
                    if i == 0:
                        ax[i, col].set_title('Condition')
                        ax[i, col].legend(bbox_to_anchor=(0.5,1.02), loc='lower center',
                                          borderaxespad=2.5)
                elif col == 5:
                    trialerror_vals = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
                    trialerror_labels = ['hit',
                                         'miss',
                                         'neutral correct reject',
                                         'neutral false alarm',
                                         'minus correct reject',
                                         'minus false alarm',
                                         'blank correct reject',
                                         'blank false alarm',
                                         'pav early licking',
                                         'pav late licking',]
                    for k in trialerror_vals:
                        ax[i, col].plot(trial_num[trialerror == trialerror_vals[k]],
                                        U.factors[2][trialerror == trialerror_vals[k], i], 'o',
                                        label=str(trialerror_labels[k]), alpha=0.8, markersize=2)
                    if i == 0:
                        ax[i, col].set_title('Trialerror')
                        ax[i, col].legend(bbox_to_anchor=(0.5, 1.02), loc='lower center',
                                          borderaxespad=2.5)

                elif col == 6:
                    h_vals = ['hungry', 'sated', 'disengaged']
                    h_labels = ['hungry', 'sated', 'disengaged']
                    color_vals = [[1, 0.6, 0.3, alpha], [0.7, 0.9, 0.4, alpha],
                                  [0.6, 0.5, 0.6, alpha], [0.0, 0.9, 0.4, alpha]]
                    for k in range(0, 3):
                        ax[i, col].plot(trial_num[hunger == h_vals[k]],
                                        U.factors[2][hunger == h_vals[k], i], 'o',
                                        label=str(h_labels[k]), color=color_vals[k], markersize=2)
                    if i == 0:
                        ax[i, col].set_title('State')
                        ax[i, col].legend(bbox_to_anchor=(0.5, 1.02), loc='lower center',
                                          borderaxespad=2.5)

                # set axes labels
                ax[i, col].set_yticks(y_ticks)
                ax[i, col].set_yticklabels(y_tickl)
                ax[i, col].set_ylim(y_lim)
                ax[i, col].set_xlim(x_lim)

                # format axes
                ax[i, col].locator_params(nbins=4)
                ax[i, col].spines['top'].set_visible(False)
                ax[i, col].spines['right'].set_visible(False)
                ax[i, col].xaxis.set_tick_params(direction='out')
                ax[i, col].yaxis.set_tick_params(direction='out')
                ax[i, col].yaxis.set_ticks_position('left')
                ax[i, col].xaxis.set_ticks_position('bottom')

                # remove xticks on all but bottom row
                if i + 1 != U.rank:
                    plt.setp(ax[i, col].get_xticklabels(), visible=False)

                if col == 3:
                    ax[i, 0].set_ylabel('Component #' + str(i+1), rotation=0,
                                        labelpad=45, verticalalignment='center',
                                        fontstyle='oblique')

        if filetype.lower() == 'pdf':
            suffix = '.pdf'
        elif filetype.lower() == 'eps':
            suffix = '.eps'
        else:
            suffix = '.png'
        plt.savefig(os.path.join(date_dir, 'rank_' + str(int(r)) + suffix),
                                 bbox_inches='tight')
        if verbose:
            plt.show()
        plt.close()


/Users/kelly_mcguire/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kelly_mcguire/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:70: RuntimeWarning: All-NaN axis encountered
/Users/kelly_mcguire/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kelly_mcguire/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:70: RuntimeWarning: All-NaN axis encountered
/Users/kelly_mcguire/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of